In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cache_dir = "/shared/3/projects/hiatus/EVAL_wegmann/cache/huggingface"
os.environ["TRANSFORMERS_CACHE"] = cache_dir
os.environ["HF_DATASETS_CACHE"] = cache_dir
output_folder = "/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/"

In [2]:
import wandb

# Login to WandB account (this might prompt for an API key if not logged in already)
wandb.login(key="c042d6be624a66d40b7f2a82a76e343896608cf0")
# Initialize a new run with a project name
wandb.init(project="bert-tiny-pretraining", entity="annawegmann")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: annawegmann (sadiri-michigan). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/annaweg/.netrc
wandb: Currently logged in as: annawegmann. Use `wandb login --relogin` to force relogin


In [3]:
# loading dataset, following https://huggingface.co/blog/pretraining-bert#4-pre-train-bert-on-habana-gaudi
from datasets import concatenate_datasets, load_dataset

bookcorpus = load_dataset("bookcorpus", split="train")
wiki = load_dataset("wikipedia", "20220301.en", split="train")
wiki = wiki.remove_columns([col for col in wiki.column_names if col != "text"])  # only keep the 'text' column

assert bookcorpus.features.type == wiki.features.type
raw_datasets = concatenate_datasets([bookcorpus, wiki])
raw_datasets = raw_datasets.shuffle(seed=42)  # You can set a seed for reproducibility

# full_dataset = load_dataset("wikitext", "wikitext-103-raw-v1")  # (before)

/opt/anaconda/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [10]:
len(raw_datasets)

80462898

In [14]:
full_dataset = load_dataset("wikitext", "wikitext-103-raw-v1") 

In [4]:
from datasets import load_dataset, DatasetDict, Dataset

# Create a new DatasetDict to store the truncated datasets
dataset = DatasetDict()

# Truncate each split to the first 10 entries and store it in the new DatasetDict
# for split, ds in raw_datasets.items():
#    dataset[split] = ds.select(range(512))
dataset = raw_datasets.select(range(512))  # ["train"]

In [5]:
# train a tokenizer
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
tokenizer = Tokenizer(BPE(unk_token="[UNK]", sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]", mask_token="[MASK]"))
trainer = BpeTrainer(vocab_size=30000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(dataset["train"]["text"], trainer=trainer)
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]'})

KeyError: "Column train not in the dataset. Current columns in the dataset: ['text']"

In [6]:
from transformers import BertForMaskedLM, RobertaTokenizer, BertConfig, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'mask_token': '[MASK]'})


# Define a lightweight BERT configuration
config = BertConfig(
    vocab_size=tokenizer.vocab_size,  # Standard BERT vocabulary size
    hidden_size=128,   # Reduced hidden layer size
    num_hidden_layers=2,  # Fewer layers
    num_attention_heads=2,  # Fewer attention heads, H/64
    intermediate_size=512  # Size of the intermediate (feed-forward) layer, 4H
)
# model = BertForMaskedLM(config=config)

model = BertForMaskedLM.from_pretrained('prajjwal1/bert-tiny') # sequence len still 512
model.resize_token_embeddings(len(tokenizer))

/opt/anaconda/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/opt/anaconda/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Embedding(50267, 128)

In [7]:
# Tokenize and prepare the dataset
def tokenize_and_encode(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)
# Apply tokenization and encoding
tokenized_datasets = dataset.map(tokenize_and_encode, batched=True, remove_columns=["text"])

In [8]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
from transformers import Trainer, TrainingArguments

# Training arguments
training_args = TrainingArguments(
    output_dir=output_folder + "bert-tiny-pretrained",
    overwrite_output_dir=True,
    max_steps=100,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir=output_folder + 'logs',
    logging_steps=500,
    report_to="wandb",  # Enables WandB integration
)

# Initialize the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# Start pretraining
trainer.train()

/opt/anaconda/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss


TrainOutput(global_step=100, training_loss=9.577440185546875, metrics={'train_runtime': 12.3172, 'train_samples_per_second': 259.8, 'train_steps_per_second': 8.119, 'total_flos': 4559683584000.0, 'train_loss': 9.577440185546875, 'epoch': 6.25})

In [32]:
# Save the trained model
trainer.save_model(output_folder + "bert-tiny-pretrained/roberta-base")
tokenizer.save_pretrained(output_folder + "bert-tiny-pretrained/roberta-base")

('/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/tokenizer_config.json',
 '/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/special_tokens_map.json',
 '/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/vocab.json',
 '/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/merges.txt',
 '/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/added_tokens.json',
 '/shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained/roberta-base/tokenizer.json')

In [30]:
import sys
# add STEL folder to path
sys.path.append('../../STEL/src/')
import torch

from STEL import STEL
from STEL.similarity import Similarity, cosine_sim
from sentence_transformers import SentenceTransformer

class SBERTSimilarity(Similarity):
    def __init__(self):
        super().__init__()
        self.model = SentenceTransformer(output_folder + "bert-tiny-pretrained")
        self.model.to("cuda" if torch.cuda.is_available() else "cpu")

    def similarities(self, sentences_1, sentences_2):
        with torch.no_grad():
            sentence_emb_1 = self.model.encode(sentences_1, show_progress_bar=False)
            sentence_emb_2 = self.model.encode(sentences_2, show_progress_bar=False)
        return [cosine_sim(sentence_emb_1[i], sentence_emb_2[i]) for i in range(len(sentences_1))]


In [31]:
STEL.eval_on_STEL(style_objects=[SBERTSimilarity()])
print()

2024-05-10 17:53:22,865 : INFO : Load pretrained SentenceTransformer: /shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained
2024-05-10 17:53:22,868 : WARNING : No sentence-transformers model found with name /shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained. Creating a new one with MEAN pooling.
Some weights of BertModel were not initialized from the model checkpoint at /shared/3/projects/hiatus/EVAL_wegmann/tiny-BERT/bert-tiny-pretrained and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-05-10 17:53:22,971 : INFO : Use pytorch device: cuda
2024-05-10 17:53:23,015 : INFO : Running STEL framework 
2024-05-10 17:53:23,030 : INFO : Filtering out tasks with low agreement ... 
2024-05-10 17:53:23

Performance on original STEL tasks


2024-05-10 17:53:29,599 : INFO : random assignments: 0
2024-05-10 17:53:29,604 : INFO :   Accuracy at 0.5596244131455399, without random 0.5596244131455399 with 0 questions
2024-05-10 17:53:29,606 : INFO :   Accuracy formality at 0.5214723926380368 for 815 task instances, without random 0.5214723926380368 with 815 left questions
2024-05-10 17:53:29,609 : INFO :   Accuracy simplicity at 0.5361963190184049 for 815 task instances, without random 0.5361963190184049 with 815 left questions
2024-05-10 17:53:29,611 : INFO :   Accuracy nbr_substitution at 0.6 for 100 task instances, without random 0.6 with 100 left questions
2024-05-10 17:53:29,612 : INFO :   Accuracy contraction at 0.65 for 100 task instances, without random 0.65 with 100 left questions
2024-05-10 17:53:29,614 : INFO :   Accuracy emotives at 0.7 for 200 task instances, without random 0.7 with 200 left questions
2024-05-10 17:53:29,616 : INFO :   Accuracy em-subj-pronoun at 0.65 for 100 task instances, without random 0.65 with

        Model Name  Accuracy  Accuracy formality  Accuracy simplicity  \
0  SBERTSimilarity  0.559624            0.521472             0.536196   

   Accuracy nbr_substitution  Accuracy contraction  Accuracy emotives  \
0                        0.6                  0.65                0.7   

   Accuracy em-subj-pronoun  
0                      0.65  
Performance on STEL-Or-Content tasks


2024-05-10 17:53:33,094 : INFO : random assignments: 0
2024-05-10 17:53:33,101 : INFO :   Accuracy at 0.09342723004694836, without random 0.09342723004694836 with 0 questions
2024-05-10 17:53:33,104 : INFO :   Accuracy formality at 0.22085889570552147 for 815 task instances, without random 0.22085889570552147 with 815 left questions
2024-05-10 17:53:33,107 : INFO :   Accuracy simplicity at 0.011042944785276074 for 815 task instances, without random 0.011042944785276074 with 815 left questions
2024-05-10 17:53:33,109 : INFO :   Accuracy nbr_substitution at 0.05 for 100 task instances, without random 0.05 with 100 left questions
2024-05-10 17:53:33,111 : INFO :   Accuracy contraction at 0.0 for 100 task instances, without random 0.0 with 100 left questions
2024-05-10 17:53:33,114 : INFO :   Accuracy emotives at 0.02 for 200 task instances, without random 0.02 with 200 left questions
2024-05-10 17:53:33,116 : INFO :   Accuracy em-subj-pronoun at 0.01 for 100 task instances, without random

        Model Name  Accuracy  Accuracy formality  Accuracy simplicity  \
0  SBERTSimilarity  0.093427            0.220859             0.011043   

   Accuracy nbr_substitution  Accuracy contraction  Accuracy emotives  \
0                       0.05                   0.0               0.02   

   Accuracy em-subj-pronoun  
0                      0.01  

